## Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Read CSV

In [2]:
raw_data = pd.read_csv("bekasi-house.csv")
raw_data.head()

,id,url,price,description
0,hos14357221,https://www.rumah123.com/properti/bekasi/hos14...,"Rp 1,35 Miliar","{'K. Tidur': '3', 'K. Mandi': '3', 'L. Tanah':..."
1,hos13901815,https://www.rumah123.com/properti/bekasi/hos13...,Rp 995 Juta,"{'K. Tidur': '3', 'K. Mandi': '3', 'L. Tanah':..."
2,hos14186873,https://www.rumah123.com/properti/bekasi/hos14...,"Rp 1,02 Miliar","{'K. Tidur': '2', 'K. Mandi': '2', 'L. Tanah':..."
3,hos14314243,https://www.rumah123.com/properti/bekasi/hos14...,"Rp 1,5 Miliar","{'K. Tidur': '3', 'K. Mandi': '2', 'L. Tanah':..."
4,hos13940617,https://www.rumah123.com/properti/bekasi/hos13...,"Rp 1,1 Miliar","{'K. Tidur': '2', 'K. Mandi': '2', 'L. Tanah':..."


In [3]:
raw_data.tail()

,id,url,price,description
1007,hos14240769,https://www.rumah123.com/properti/bekasi/hos14...,"Rp 1,29 Miliar","{'K. Tidur': '3', 'K. Mandi': '2', 'L. Tanah':..."
1008,hos14240715,https://www.rumah123.com/properti/bekasi/hos14...,Rp 490 Juta,"{'K. Tidur': '2', 'K. Mandi': '1', 'L. Tanah':..."
1009,hos14240713,https://www.rumah123.com/properti/bekasi/hos14...,"Rp 1,2 Miliar","{'K. Tidur': '2', 'K. Mandi': '1', 'L. Tanah':..."
1010,hos14240640,https://www.rumah123.com/properti/bekasi/hos14...,Rp 660 Juta,"{'K. Tidur': '2', 'K. Mandi': '1', 'L. Tanah':..."
1011,hos14240623,https://www.rumah123.com/properti/bekasi/hos14...,"Rp 1,25 Miliar","{'K. Tidur': '3', 'K. Mandi': '2', 'L. Tanah':..."


In [4]:
# Check the data types
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1012 non-null   object
 1   url          1012 non-null   object
 2   price        1012 non-null   object
 3   description  1012 non-null   object
dtypes: object(4)
memory usage: 31.8+ KB


In [5]:
# Check if there is any missing values in raw_data by each column
raw_data.isnull().sum()

id             0
url            0
price          0
description    0
dtype: int64

If you're interested by house data in Bekasi, you can check all of them by using **web-scraping.py**. In this case, we want to do data preprocessing. As you can see, our raw_data is not cleaned, for example "description" column is object-typed, also it seems that *all inputs can be converted each value to be dictionary*.
<br><br>
So, let's do that!

## Data Preprocessing

In [6]:
# Since we wanted to solve house prediction problem, therefore this is supervised learning. 
# We want to only include all the relevant inputs
data = raw_data.copy()

# Then, we will not include "id" and "url" column
data.drop(["id", "url"], axis=1, inplace=True)
data.head()

,price,description
0,"Rp 1,35 Miliar","{'K. Tidur': '3', 'K. Mandi': '3', 'L. Tanah':..."
1,Rp 995 Juta,"{'K. Tidur': '3', 'K. Mandi': '3', 'L. Tanah':..."
2,"Rp 1,02 Miliar","{'K. Tidur': '2', 'K. Mandi': '2', 'L. Tanah':..."
3,"Rp 1,5 Miliar","{'K. Tidur': '3', 'K. Mandi': '2', 'L. Tanah':..."
4,"Rp 1,1 Miliar","{'K. Tidur': '2', 'K. Mandi': '2', 'L. Tanah':..."


### Price

Rupiah (Rp) is the official currency in Indonesia.

In [7]:
# Delete the 'Rp ' text from price column, but don't forget that the price is in 'Rp' currency
clean_price = data['price'].str.replace("Rp ", "")
clean_price

0       1,35 Miliar
1          995 Juta
2       1,02 Miliar
3        1,5 Miliar
4        1,1 Miliar
           ...     
1007    1,29 Miliar
1008       490 Juta
1009     1,2 Miliar
1010       660 Juta
1011    1,25 Miliar
Name: price, Length: 1012, dtype: object

In [8]:
# We want to split the clean_price variable to get "price amount" and "price unit"
price_amt = clean_price.apply(lambda x: x.split()[0])
price_unit = clean_price.apply(lambda x: x.split()[1])

# See all units in bekasi-housing price
set(price_unit)

{'Juta', 'Miliar'}

* **Juta** *(Indonesian)* means **millions**
* **Miliar** *(Indonesian)* means **billions**

#### Price Amount

In [9]:
# Random 10 Price Amount
list(price_amt)[:10]

['1,35', '995', '1,02', '1,5', '1,1', '1,1', '1,25', '495', '363', '1,55']

In [10]:
# Make price amount to be numerical (float)
price_amt = price_amt.str.replace(",", ".").astype(float)

price_amt[:10]

0      1.35
1    995.00
2      1.02
3      1.50
4      1.10
5      1.10
6      1.25
7    495.00
8    363.00
9      1.55
Name: price, dtype: float64

#### Price Unit

In [11]:
# Remember:
# 1. Juta: Millions (10^6)
# 2. Miliar: Billions (10^9)

# Convert the price unit in billions rupiah unit (from string to float)
price_unit = price_unit.map({"Juta": 1e-3, "Miliar": 1})
price_unit[:10]

0    1.000
1    0.001
2    1.000
3    1.000
4    1.000
5    1.000
6    1.000
7    0.001
8    0.001
9    1.000
Name: price, dtype: float64

In [12]:
clean_price = price_amt * price_unit
clean_price[:10]

0    1.350
1    0.995
2    1.020
3    1.500
4    1.100
5    1.100
6    1.250
7    0.495
8    0.363
9    1.550
Name: price, dtype: float64

In [13]:
data['price'] = clean_price
data.head()

,price,description
0,1.350,"{'K. Tidur': '3', 'K. Mandi': '3', 'L. Tanah':..."
1,0.995,"{'K. Tidur': '3', 'K. Mandi': '3', 'L. Tanah':..."
2,1.020,"{'K. Tidur': '2', 'K. Mandi': '2', 'L. Tanah':..."
3,1.500,"{'K. Tidur': '3', 'K. Mandi': '2', 'L. Tanah':..."
4,1.100,"{'K. Tidur': '2', 'K. Mandi': '2', 'L. Tanah':..."


### Description

In [14]:
import json

# Replace ' by " as json module expects all the properties are enclosed in double quotes
preprocessed_inputs = data['description'].str.replace("'", '"')
inputs_list = [json.loads(dict_item) for dict_item in list(preprocessed_inputs)]

inputs_list[0]

{'K. Tidur': '3',
 'K. Mandi': '3',
 'L. Tanah': '77 m²',
 'L. Bangunan': '112 m²',
 'Carport': '2',
 'Tipe Properti': 'Rumah',
 'Sertifikat': 'SHM - Sertifikat Hak Milik',
 'Daya Listrik': '2200 Watt',
 'KT. Pembantu': '1',
 'Jumlah Lantai': '2',
 'Tahun dibangun': '2021',
 'Kondisi Properti': 'Bagus',
 'Kondisi Perabotan': 'Semi Furnished',
 'ID Iklan': 'hos14357221'}

#### Get all unique keys

In [15]:
# Obtain all the unique keys in the data
keys = []
for dict_item in inputs_list:
    for key in dict_item.keys():
        if key not in keys:
            keys.append(key)

In [16]:
# Print all keys
print("List of Unique Keys: \n")
for i in range(len(keys)):
    print(f"{i+1}. {keys[i]}")

List of Unique Keys: 

1. K. Tidur
2. K. Mandi
3. L. Tanah
4. L. Bangunan
5. Carport
6. Tipe Properti
7. Sertifikat
8. Daya Listrik
9. KT. Pembantu
10. Jumlah Lantai
11. Tahun dibangun
12. Kondisi Properti
13. Kondisi Perabotan
14. ID Iklan
15. Hadap
16. Garasi
17. KM. Pembantu


In [17]:
# Input all the keys and values in inputs_dict
inputs_dict = {key:[inputs[key] if key in inputs.keys() else None for inputs in inputs_list] for key in keys}

# Choose any column to see its first 10 values, for example: "K. Tidur" (bedroom amount)
column = "K. Tidur"
inputs_dict[column][:10]

['3', '3', '2', '3', '2', '3', '3', '2', '1', '3']

In [18]:
# Convert the dictionary data into DataFrame
inputs = pd.DataFrame(data=inputs_dict)
inputs.head()

,K. Tidur,K. Mandi,L. Tanah,L. Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,KT. Pembantu,Jumlah Lantai,Tahun dibangun,Kondisi Properti,Kondisi Perabotan,ID Iklan,Hadap,Garasi,KM. Pembantu
0,3,3,77 m²,112 m²,2,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,2,2021,Bagus,Semi Furnished,hos14357221,None,None,None
1,3,3,73 m²,93 m²,None,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,2023,Baru,Unfurnished,hos13901815,Timur,None,None
2,2,2,105 m²,90 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,None,Sudah Renovasi,Unfurnished,hos14186873,Selatan,2,None
3,3,2,210 m²,150 m²,None,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,1,None,Bagus,Unfurnished,hos14314243,None,None,1
4,2,2,60 m²,50 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,None,Bagus,Unfurnished,hos13940617,Utara,None,None


In [19]:
# Concatenate price data and inputs data
price_df = data[['price']].copy()
cleaned_data = pd.concat([inputs, price_df], axis=1)

cleaned_data.head()

,K. Tidur,K. Mandi,L. Tanah,L. Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,KT. Pembantu,Jumlah Lantai,Tahun dibangun,Kondisi Properti,Kondisi Perabotan,ID Iklan,Hadap,Garasi,KM. Pembantu,price
0,3,3,77 m²,112 m²,2,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,2,2021,Bagus,Semi Furnished,hos14357221,None,None,None,1.350
1,3,3,73 m²,93 m²,None,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,2023,Baru,Unfurnished,hos13901815,Timur,None,None,0.995
2,2,2,105 m²,90 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,None,Sudah Renovasi,Unfurnished,hos14186873,Selatan,2,None,1.020
3,3,2,210 m²,150 m²,None,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,1,None,Bagus,Unfurnished,hos14314243,None,None,1,1.500
4,2,2,60 m²,50 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,None,Bagus,Unfurnished,hos13940617,Utara,None,None,1.100


## Checkpoint

1. Rename columns
2. Data Types
3. Rename values

### Rename columns

In [20]:
new_columns = ["bedroom", "bathroom", "land_area", "building_area", "carport", "prop_type", 
               "certificate", "electric_power", "maid_bedroom", "floor_num", "year_built", "prop_cond",
               "furniture_cond", "ad_id", "facing", "garage", "maid_bathroom", "price"]
cleaned_data.columns = new_columns

cleaned_data.head()

,bedroom,bathroom,land_area,building_area,carport,prop_type,certificate,electric_power,maid_bedroom,floor_num,year_built,prop_cond,furniture_cond,ad_id,facing,garage,maid_bathroom,price
0,3,3,77 m²,112 m²,2,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,2,2021,Bagus,Semi Furnished,hos14357221,None,None,None,1.350
1,3,3,73 m²,93 m²,None,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,2023,Baru,Unfurnished,hos13901815,Timur,None,None,0.995
2,2,2,105 m²,90 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,None,Sudah Renovasi,Unfurnished,hos14186873,Selatan,2,None,1.020
3,3,2,210 m²,150 m²,None,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,1,None,Bagus,Unfurnished,hos14314243,None,None,1,1.500
4,2,2,60 m²,50 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,None,2,None,Bagus,Unfurnished,hos13940617,Utara,None,None,1.100


### Data Values Replacement

In [21]:
cleaned_data.dtypes

bedroom            object
bathroom           object
land_area          object
building_area      object
carport            object
prop_type          object
certificate        object
electric_power     object
maid_bedroom       object
floor_num          object
year_built         object
prop_cond          object
furniture_cond     object
ad_id              object
facing             object
garage             object
maid_bathroom      object
price             float64
dtype: object

In [22]:
# Land Area
cleaned_data['land_area'] = cleaned_data['land_area'].str.replace("m²", "").str.strip()

# Building Area
cleaned_data['building_area'] = cleaned_data['building_area'].str.replace("m²", "").str.strip()

# Property Type
cleaned_data['prop_type'] = cleaned_data['prop_type'].map({'Rumah': 'House'})

# Electric Power
cleaned_data['electric_power'] = cleaned_data['electric_power'].str.replace('Watt', '').str.strip().str.lower()

# Property Condition
cleaned_data['prop_cond'] = cleaned_data['prop_cond'].str.title().map({"Bagus": "Good", 
                                                                       "Baru": "New", 
                                                                       "Sudah Renovasi": "Renovated",
                                                                       "Butuh Renovasi": "Needs Renovation",
                                                                       "Semi Furnished": "Semi Furnished"
                                                                      })

# Furniture Condition
cleaned_data['furniture_cond'] = cleaned_data['furniture_cond'].str.title()

# Facing
cleaned_data['facing'] = cleaned_data['facing'].map({"Timur": "East",
                                                     "Selatan": "South",
                                                     "Utara": "North",
                                                     "Barat": "West",
                                                     "Timur Laut": "Northeast",
                                                     "Barat Laut": "Northwest",
                                                     "Tenggara": "Southeast",
                                                     "Barat Daya": "Southwest"
                                                    })

# Replace None to NaN value
cleaned_data = cleaned_data.fillna(np.nan)

cleaned_data.head()

,bedroom,bathroom,land_area,building_area,carport,prop_type,certificate,electric_power,maid_bedroom,floor_num,year_built,prop_cond,furniture_cond,ad_id,facing,garage,maid_bathroom,price
0,3,3,77,112,2,House,SHM - Sertifikat Hak Milik,2200,1,2,2021,Good,Semi Furnished,hos14357221,NaN,NaN,NaN,1.350
1,3,3,73,93,NaN,House,SHM - Sertifikat Hak Milik,2200,NaN,2,2023,New,Unfurnished,hos13901815,East,NaN,NaN,0.995
2,2,2,105,90,1,House,SHM - Sertifikat Hak Milik,2200,NaN,2,NaN,Renovated,Unfurnished,hos14186873,South,2,NaN,1.020
3,3,2,210,150,NaN,House,SHM - Sertifikat Hak Milik,2200,1,1,NaN,Good,Unfurnished,hos14314243,NaN,NaN,1,1.500
4,2,2,60,50,1,House,SHM - Sertifikat Hak Milik,2200,NaN,2,NaN,Good,Unfurnished,hos13940617,North,NaN,NaN,1.100


### Change Data Types

#### Numerical Features

In [32]:
# bedroom, bathroom, land_area, building_area, maid_bedroom, maid_bathroom, electric_power -> float
num_columns = ["bedroom", "carport", "bathroom", "land_area", "building_area", "maid_bedroom", "maid_bathroom", "year_built", "floor_num", "garage", "electric_power"]
for column in num_columns:
    cleaned_data[column] = cleaned_data[column].apply(pd.to_numeric, errors='coerce')

cleaned_data[num_columns].dtypes

bedroom           float64
carport           float64
bathroom          float64
land_area         float64
building_area     float64
maid_bedroom      float64
maid_bathroom     float64
year_built        float64
floor_num         float64
garage            float64
electric_power    float64
dtype: object

#### Categorical Features and Target Data

In [33]:
cat_columns = [col for col in cleaned_data.columns if col not in num_columns]

cleaned_data[cat_columns].dtypes

prop_type          object
certificate        object
prop_cond          object
furniture_cond     object
ad_id              object
facing             object
price             float64
dtype: object

### Check Missing Values

In [35]:
# Check missing values in categorical features
cleaned_data[cat_columns].isnull().sum()

prop_type           0
certificate         3
prop_cond          46
furniture_cond    183
ad_id               0
facing            587
price               0
dtype: int64

In [36]:
# Check missing values in numerical features
cleaned_data[num_columns].isnull().sum()

bedroom            24
carport           266
bathroom           21
land_area           0
building_area       0
maid_bedroom      770
maid_bathroom     857
year_built        533
floor_num           9
garage            743
electric_power     54
dtype: int64

### Another Data Preprocessing Step

In [38]:
# Fill missing values in numerical features only, except year_built; since we can use simple imputation
for col in num_columns:
    if col != "year_built":
        cleaned_data[col] = cleaned_data[col].fillna(0)
        
cleaned_data.head()

,bedroom,bathroom,land_area,building_area,carport,prop_type,certificate,electric_power,maid_bedroom,floor_num,year_built,prop_cond,furniture_cond,ad_id,facing,garage,maid_bathroom,price
0,3.0,3.0,77.0,112.0,2.0,House,SHM - Sertifikat Hak Milik,2200.0,1.0,2.0,2021.0,Good,Semi Furnished,hos14357221,NaN,0.0,0.0,1.350
1,3.0,3.0,73.0,93.0,0.0,House,SHM - Sertifikat Hak Milik,2200.0,0.0,2.0,2023.0,New,Unfurnished,hos13901815,East,0.0,0.0,0.995
2,2.0,2.0,105.0,90.0,1.0,House,SHM - Sertifikat Hak Milik,2200.0,0.0,2.0,NaN,Renovated,Unfurnished,hos14186873,South,2.0,0.0,1.020
3,3.0,2.0,210.0,150.0,0.0,House,SHM - Sertifikat Hak Milik,2200.0,1.0,1.0,NaN,Good,Unfurnished,hos14314243,NaN,0.0,1.0,1.500
4,2.0,2.0,60.0,50.0,1.0,House,SHM - Sertifikat Hak Milik,2200.0,0.0,2.0,NaN,Good,Unfurnished,hos13940617,North,0.0,0.0,1.100


## Save New CSV File

In [39]:
cleaned_data.to_csv("bekasi-house-new.csv")